In [20]:
# Libraries to help with reading and manipulating data
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.utils import resample
import pandas as pd
import numpy as np

from datetime import datetime

# Libaries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# To tune model, get different metric scores, and split data
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn import metrics

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# To be used for data scaling and one hot encoding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# To impute missing values
from sklearn.impute import SimpleImputer

# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# To do hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

# To be used for creating pipelines and personalizing them
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To help with model building
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

# To suppress scientific notations
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [30]:
pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.4 MB/s eta 0:00:00


In [4]:
train_data = pd.read_csv('train_waves.csv') ##  read the data

test_data = pd.read_csv('test_waves.csv') ##  read the data


In [5]:
train_data.head()


,ID,#YY,MM,DD,hh,mm,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WVHT(m)
0,2017424150,2017,4,24,1,50,23,3.000,3.600,4.170,3.330,349,1019.100,4.700,5.100,4.200,0.390
1,2017424250,2017,4,24,2,50,30,2.900,3.300,5.260,4.210,358,1018.800,4.600,4.800,4.100,0.650
2,2017424350,2017,4,24,3,50,360,2.200,2.500,5.880,4.790,11,1018.300,4.600,4.700,4.200,0.900
3,2017424450,2017,4,24,4,50,355,2.500,3.100,5.880,4.950,23,1018.100,4.600,4.600,4.200,0.910
4,2017424550,2017,4,24,5,50,355,3.500,4.500,6.250,4.980,26,1018.300,4.500,4.600,4.000,0.880


In [6]:
df=train_data.copy()


In [25]:
"""
# Calculate the sine and cosine of wind direction
df['WDIR_sin'] = np.sin(np.radians(df['WDIR(degT)']))
df['WDIR_cos'] = np.cos(np.radians(df['WDIR(degT)']))
#wind speed cubed
df['WSCB']=np.power(df['WSPD(m/s)'],3)
#dominant wind power squared
df['DPD2']=np.power(df['DPD(sec)'],2)
#temperature differene between air and water
df['TDIF']=df['ATMP(degC)']-df['WTMP(degC)']
test_data['WDIR_sin'] = np.sin(np.radians(test_data['WDIR(degT)']))
test_data['WDIR_cos'] = np.cos(np.radians(test_data['WDIR(degT)']))
test_data['WSCB']=np.power(test_data['WSPD(m/s)'],3)
test_data['DPD2']=np.power(test_data['DPD(sec)'],2)
test_data['TDIF']=test_data['ATMP(degC)']-test_data['WTMP(degC)']

SyntaxError: incomplete input (<ipython-input-25-cb3ec1674ebc>, line 1)

calculating the sine and cosine: Instead of using the raw wind direction feature, it may be helpful to convert it into its sine and cosine components to capture any directional patterns in the wind that affect wave height.

In [26]:
train_data.shape


(3886, 17)

In [27]:
X=train_data.iloc[:,6:16]
X.head()

,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC)
0,23,3.000,3.600,4.170,3.330,349,1019.100,4.700,5.100,4.200
1,30,2.900,3.300,5.260,4.210,358,1018.800,4.600,4.800,4.100
2,360,2.200,2.500,5.880,4.790,11,1018.300,4.600,4.700,4.200
3,355,2.500,3.100,5.880,4.950,23,1018.100,4.600,4.600,4.200
4,355,3.500,4.500,6.250,4.980,26,1018.300,4.500,4.600,4.000


In [28]:
y=train_data.iloc[:,-1]
y.head()

0   0.390
1   0.650
2   0.900
3   0.910
4   0.880
Name: WVHT(m), dtype: float64

In [31]:
from catboost import CatBoostRegressor
model=CatBoostRegressor()
model.fit(X,y)

Learning rate set to 0.050737
0:	learn: 0.3575044	total: 57.1ms	remaining: 57.1s
1:	learn: 0.3427556	total: 65.7ms	remaining: 32.8s
2:	learn: 0.3288714	total: 71.9ms	remaining: 23.9s
3:	learn: 0.3152773	total: 81.7ms	remaining: 20.3s
4:	learn: 0.3023711	total: 94.2ms	remaining: 18.7s
5:	learn: 0.2906837	total: 104ms	remaining: 17.2s
6:	learn: 0.2790179	total: 111ms	remaining: 15.8s
7:	learn: 0.2683194	total: 116ms	remaining: 14.4s
8:	learn: 0.2584242	total: 127ms	remaining: 14s
9:	learn: 0.2486330	total: 133ms	remaining: 13.2s
10:	learn: 0.2396784	total: 140ms	remaining: 12.6s
11:	learn: 0.2313149	total: 150ms	remaining: 12.3s
12:	learn: 0.2230305	total: 156ms	remaining: 11.9s
13:	learn: 0.2157048	total: 163ms	remaining: 11.5s
14:	learn: 0.2083039	total: 172ms	remaining: 11.3s
15:	learn: 0.2013110	total: 182ms	remaining: 11.2s
16:	learn: 0.1948457	total: 191ms	remaining: 11.1s
17:	learn: 0.1884321	total: 197ms	remaining: 10.7s
18:	learn: 0.1826164	total: 204ms	remaining: 10.5s
19:	lear

In [32]:
X_test=test_data.iloc[:,6:16]
X_test.head()

,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC)
0,340,6.700,8.200,2.350,2.370,177,1011.500,18.600,16.800,17.800
1,15,2.300,5.500,2.600,2.440,336,1010.700,19.000,16.900,17.800
2,11,1.600,2.700,3.130,2.950,341,1010.600,17.900,16.700,17.300
3,21,4.100,4.900,2.860,2.590,8,1015.800,20.000,20.300,19.500
4,16,4.500,5.400,2.740,2.640,16,1016.500,20.100,20.300,19.600


In [33]:
pred=model.predict(X_test)


In [34]:
pred

array([0.28880797, 0.35064786, 0.36220956, ..., 0.73227945, 1.22184061,
       1.24041024])

In [35]:
output=pd.DataFrame({'Id':test_data.ID,'WVHT(m)':pred})


In [36]:
output.head()

,Id,WVHT(m)
0,2019722240,0.289
1,2019722340,0.351
2,201973140,0.362
3,2019761040,0.247
4,2019761140,0.256


In [46]:
from sklearn.utils import resample

# Assuming y is your target vector
# Example:
# y = pd.read_csv('your_target.csv')

# Select a random sample of 1535 values from y
y_sample = resample(y, n_samples=1535, random_state=42)


In [47]:
rmse = np.sqrt(mean_squared_error(y_sample, pred))
print("{}".format( rmse))

0.4980995935258169
